# Drive Mounting

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
cd drive/"My Drive"/Pala_Project/Keras Network

In [0]:
ls

# Data Network Preparation

In [0]:
import random
from keras.preprocessing.image import ImageDataGenerator

''' **** Data Augmentation **** '''

def flip_img(img, depth, traj):
  flip_img = cv.flip(img, 1)
  flip_depth = cv.flip(depth, 1)
  flip_traj = [[IMAGE_WIDTH - 1 - x, y] for x, y in traj]
  return flip_img, flip_depth, flip_traj

def randomBrightness(img):
  
  # random.seed()
  
  dictForGenerator = {"brightness": random.uniform(0.4, 2)}
  
  img_gen = ImageDataGenerator()
  img = img_gen.apply_transform(img, dictForGenerator)
  
  return img

In [0]:
import pandas as pd
import numpy as np
import cv2 as cv

def batch_generator(images_paths, trajectories, batch_size):
    """
    Generate training images given image paths and associated images trajectories
    """
    
    # np.random.seed()
    
    images = np.zeros([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, 4]) # IMAGE_CHANNELS = 4 Channels (R, G, B, D)
    # images = np.zeros([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, 3]) # IMAGE_CHANNELS = 3 Channels (R, G, B)
    pos_vect = np.zeros([batch_size, 30, 2])
    
    while True:
        i = 0
        for index in np.random.permutation(images_paths.shape[0]):
            image_path = images_paths[index][0]
            depth_path = images_paths[index][1]
            
            img = cv.imread(image_path)
            
            depth = cv.imread(depth_path, -1)
            depth[depth > 10000] = 10000
            
            traj = trajectories[index]
            
            # ********  START   Data Augmentation  ********
            
            if np.random.rand() < 0.5:
              img, depth, traj = flip_img(img, depth, traj)
            if np.random.rand() < 0.5:
              img = randomBrightness(img)
              
            # ********  END     Data Augmentation  ********
            
            img = img / 127.5 - 1    # Normalizzazione tra [-1, 1]
            depth = depth / 5000 - 1 # Normalizzazione tra [-1, 1]
            
            depth = np.reshape(depth, (IMAGE_HEIGHT, IMAGE_WIDTH, 1))
            rgbd = np.concatenate((img, depth), axis=2)
            # rgbd = img # NB: Se IMAGE_CHANNELS = 3
            
            images[i] = rgbd
            pos_vect[i] = np.array(traj)
            
            i += 1
            if i == batch_size:
                break
        yield images, pos_vect
        # return images, pos_vect

def load_data(train_csv = 'CSV/train_driving_log.csv', valid_csv = 'CSV/validation_driving_log.csv'):
  train_data_df = pd.read_csv(train_csv, error_bad_lines=False)
  valid_data_df = pd.read_csv(valid_csv, error_bad_lines=False)
  
  X_train = train_data_df[['image','depth']].values
  X_valid = valid_data_df[['image','depth']].values
  
  y_train = train_data_df['trajectory'].values
  y_train = [[[int(xy.split(',')[0]), int(xy.split(',')[1])] for xy in trajectory.split(':')] for trajectory in y_train]
  
  y_valid = valid_data_df['trajectory'].values
  y_valid = [[[int(xy.split(',')[0]), int(xy.split(',')[1])] for xy in trajectory.split(':')] for trajectory in y_valid]
  
  return X_train, X_valid, y_train, y_valid

In [0]:
X_train, X_valid, y_train, y_valid = load_data()

# Keras Installation

In [0]:
!pip install keras
import keras
keras.__version__

from keras.models import Sequential, load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Reshape
import keras.backend as K

# Definition and Training of a New Network

###Definition of a new Network

In [0]:
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 4 # IMAGE_CHANNELS = 4 Channels (R, G, B, D)
# IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 3 # IMAGE_CHANNELS = 3 Channels (R, G, B)

INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

model = Sequential()
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2), input_shape=INPUT_SHAPE))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))

model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='elu'))
model.add(Dense(128, activation='elu'))
model.add(Dense(60))
model.add(Reshape((30,2)))

model.summary()

In [0]:
# Save model scheme with shapes

from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True)

In [0]:
checkpoint = ModelCheckpoint('SavedModel4/saved_model{epoch:03d}.h5', # SavedModel{n}/...
                              monitor='val_loss',
                              verbose=0,
                              save_best_only=False,
                              mode='auto',
                              period=5)

In [0]:
model.compile(loss='mean_squared_error', optimizer=Adam(lr=1.0e-4), metrics=['mae'])

###Training Phase

In [0]:
import math
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

def scheduler(epoch):
    lr = K.get_value(model.optimizer.lr)
    if (epoch + 1) % 50 == 0: # Ogni 50 epoche dimezzo il learning-rate
        K.set_value(model.optimizer.lr, lr/2)
        lr = K.get_value(model.optimizer.lr) # Nuovo valore del learning rate
        print('*** NUOVO LEARNING RATE: {lr_value} ***'.format(lr_value=lr))
    return lr

In [0]:
change_lr = LearningRateScheduler(scheduler)

bs = 20

history = model.fit_generator(batch_generator(X_train, y_train, bs),
                    steps_per_epoch=roundup(len(X_train)/bs),
                    epochs=200,
                    max_q_size=20,
                    validation_data=batch_generator(X_valid, y_valid, bs),
                    nb_val_samples=roundup(len(X_valid)/bs),
                    callbacks=[checkpoint, change_lr],
                  	workers=10,
			              use_multiprocessing=True,
                    verbose=1)

# Resume Training from checkpoint

### Loading existing Network

In [0]:
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 4 # IMAGE_CHANNELS = 4 Channels (R, G, B, D)
# IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 3 # IMAGE_CHANNELS = 3 Channels (R, G, B)

INPUT_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS)

model = load_model('SavedModel4/saved_model100.h5') # SavedModel{n}/...
model.summary()

In [0]:
checkpoint = ModelCheckpoint('SavedModel4/saved_model{epoch:03d}.h5', # SavedModel{n}/...
                              monitor='val_loss',
                              verbose=0,
                              save_best_only=False,
                              mode='auto',
                              period=5)

In [0]:
model.compile(loss='mean_squared_error', optimizer=Adam(lr=(1.0e-4)/8), metrics=['mae']) #NB: Mettere un lr coerente con l'ultimo utilizzato dalla rete prima di interrompere il training

### Training Phase Resume

In [0]:
import math
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

def scheduler(epoch):
    lr = K.get_value(model.optimizer.lr)
    if (epoch + 1) % 50 == 0: # Ogni 50 epoche dimezzo il learning-rate
        K.set_value(model.optimizer.lr, lr/2)
        lr = K.get_value(model.optimizer.lr) # Nuovo valore del learning rate
        print('*** NUOVO LEARNING RATE: {lr_value} ***'.format(lr_value=lr))
    return lr

In [0]:
change_lr = LearningRateScheduler(scheduler)

bs=20

history = model.fit_generator(batch_generator(X_train, y_train, bs),
                    steps_per_epoch=roundup(len(X_train)/bs),
                    epochs=200,
                    max_q_size=20,
                    validation_data=batch_generator(X_valid, y_valid, bs),
                    nb_val_samples=roundup(len(X_valid)/bs),
                    callbacks=[checkpoint, change_lr],
                  	workers=10,
			              use_multiprocessing=True,
                    initial_epoch=100, # NB: Se ha salvato fino a N, mettere N
                    verbose=1,)   

# Loss and MAE Graphs

In [0]:
import matplotlib.pyplot as plt
print(history.history.keys())

#### Loss Graph

In [0]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper left')

''' # Per Stampare con i corretti valori dell'asse-x quando salta la connessione
curr_x =    [  0,  20,  40,  60,  80, 100]
my_xticks = [100, 120, 140, 160, 180, 200]
plt.xticks(curr_x, my_xticks)
'''

# plt.savefig('loss4.pdf') # loss{n}.pdf
plt.savefig('loss4.png') # loss{n}.png

plt.show()

#### MAE Graph

In [0]:
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('model mean_absolute_error')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc = 'upper left')

''' # Per Stampare con i corretti valori dell'asse-x quando salta la connessione
curr_x =    [  0,  20,  40,  60,  80, 100]
my_xticks = [100, 120, 140, 160, 180, 200]
plt.xticks(curr_x, my_xticks)
'''

# plt.savefig('mae.4pdf') # mae{n}.pdf
plt.savefig('mae2.png') # mae{n}.png

plt.show()

# Validation Phase

#### evaluate_generator

In [0]:
import math
def roundup(x):
    return int(math.ceil(x / 10.0)) * 10

In [0]:
bs = 20

model = load_model('SavedModel4/saved_model100.h5') # SavedModel{n}/...

scores = model.evaluate_generator(batch_generator(X_valid, y_valid, bs),
                                  # steps = roundup(len(X_valid)/bs),
                                  steps=5,
                                  max_queue_size=10,
                                  verbose=1)

In [0]:
print(scores)

#### evaluate

In [0]:
bs = 20

model = load_model('SavedModel4/saved_model100.h5') # SavedModel{n}/...

x, y = batch_generator(X_valid, y_valid, bs) # NB: Cambiare 'yield' con 'return' in batch_generator

scores = model.evaluate(x, y, verbose=0)

In [0]:
print(scores)

# Testing Phase

In [0]:
import cv2 as cv
import numpy as np
import os

In [0]:
IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 4      # IMAGE_CHANNELS = 4 Channels (R, G, B, D)
# IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 3    # IMAGE_CHANNELS = 3 Channels (R, G, B)

n_seq = str(16) # str(17)
imgs_path   = 'Dataset/IMG/test/seq{n}/left/'.format(n=n_seq)
depths_path = 'Dataset/IMG/test/seq{n}/depth/'.format(n=n_seq)
# imgs_path   = '* Test */test/seq{n}/left/'.format(n=n_seq)
# depths_path = '* Test */test_flip/seq{n}/depth/'.format(n=n_seq)

images_list = [img   for img   in sorted(os.listdir(imgs_path))   if img.endswith('.png')]
depths_list = [depth for depth in sorted(os.listdir(depths_path)) if depth.endswith('.png')]

images_list[0:250] = [] # Tolgo i primi 250 elementi
depths_list[0:250] = [] # Tolgo i primi 250 elementi

print(images_list)

imgs = []
depths = []

for image_name in images_list:
  img = cv.imread(imgs_path + image_name)
  img = img / 127.5 - 1
  imgs.append(img)  
  print('Image: ', image_name)

for image_name in depths_list:
  depth = cv.imread(depths_path + image_name, -1)
  depth = np.reshape(depth, (IMAGE_HEIGHT, IMAGE_WIDTH, 1))
  depth[depth > 10000] = 10000
  depth = depth / 5000 - 1
  depths.append(depth)
  print('Depth: ', image_name)

In [0]:
# IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 4 # IMAGE_CHANNELS = 4 Channels (R, G, B, D)
# IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS = 80, 160, 3 # IMAGE_CHANNELS = 3 Channels (R, G, B)

dim = len(images_list)
images = np.zeros([dim, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_CHANNELS])

for i, img in enumerate(imgs):
  images[i] = np.concatenate((img, depths[i]), axis=2) # Se IMAGE_CHANNELS = 4
  # images[i] = img # Se IMAGE_CHANNELS = 3

In [0]:
from keras.models import load_model
import json

epochs = ['050', '070', '100', '150', '200']
# models = ['3', '3*', '4', '4*']
# models = ['3']
models = ['4']

for n_mod in models:
  for n_epoch in epochs:
    model = load_model('SavedModel{mod}/saved_model{epoch}.h5'.format(mod=n_mod, epoch=n_epoch))
    prediction = model.predict(images)
    print('*** Prediction Shape: ***', prediction.shape, 'Model {mod}_ep{ep}'.format(mod = n_mod, ep=n_epoch))

    file = '#{mod} prediction_seq{n}_ep{epoch}.json'.format(mod=n_mod, n=n_seq, epoch=n_epoch)
    with open(file, 'w') as f:
      json.dump(prediction.tolist(), f)

# Part of Notebook Reserved for Saving Images for the Report

In [0]:
import os, json, random
import cv2 as cv
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

def resize(img):
  SCALE_PERCENT = 0.125
  height = img.shape[0]
  width = img.shape[1]
  new_width = int(width * SCALE_PERCENT)
  new_height = int(height * SCALE_PERCENT)
  dim = (new_width, new_height) # (1280, 640) -> (160, 80)
  resized = cv.resize(img, dim, interpolation=cv.INTER_AREA)
  return resized

def flip_img(img, depth, traj):
  IMAGE_WIDTH = 160
  flip_img = cv.flip(img, 1)
  flip_depth = cv.flip(depth, 1)
  flip_traj = np.array([[IMAGE_WIDTH - 1 - x, y] for x, y in traj])
  return flip_img, flip_depth, flip_traj

def randomBrightness(img):
  # random.seed()
  dictForGenerator = {"brightness": 0.4} # random.uniform(0.4, 2)}
  img_gen = ImageDataGenerator()
  img = img_gen.apply_transform(img, dictForGenerator)
  return img

In [0]:
def saveExamplesOfImages(image_path, depth_path, trajectories_path):
    
    num_of_points = 30
    
    with open(trajectories_path) as json_file:
        data = json.load(json_file)

    original_image = cv.imread(image_path)
    original_depth = cv.imread(depth_path, -1)
    new_depth = np.array(original_depth)
    new_depth[new_depth > 10000] = 10000
    
    n_frame = int(image_path.split('/')[-1][-10:-4]) - 1
    frame = 'frame_{n}'.format(n='0' * (6 - len(str(n_frame))) + str(n_frame))

    coorFuture = data[frame]["object_0"]["future"]
    coorFuture = coorFuture[0:num_of_points] if len(coorFuture) >= num_of_points else coorFuture # Vengono mostrati solo i primi 30 punti
          
    for i, item in enumerate(coorFuture):
      y = int(item[1])
      x = int(item[0])

      cv.circle(original_image, (x, y), 3, (0, 255, 0), 2)
    
    cv.imwrite('ReportImages/DS/Image/original_image.png', original_image)
    cv.imwrite('ReportImages/DS/Depth/original_depth.png', original_depth)
    cv.imwrite('ReportImages/DS/Depth/new_depth.png', new_depth)
    cv.imwrite('ReportImages/DS/Image/cropped_image.png', original_image[60:-20, :, :])
    cv.imwrite('ReportImages/DS/Depth/cropped_depth.png', original_depth[60:-20, :])
    cv.imwrite('ReportImages/DS/Depth/cropped_new_depth.png', new_depth[60:-20, :])
    cv.imwrite('ReportImages/DS/Image/small_image.png', resize(original_image[60:-20, :, :]))
    cv.imwrite('ReportImages/DS/Depth/small_depth.png', resize(original_depth[60:-20, :]))
    
    small_new_depth = resize(original_depth[60:-20, :])
    small_new_depth[small_new_depth > 10000] = 10000
    cv.imwrite('ReportImages/DS/Depth/small_new_depth.png', small_new_depth)
    # cv.imwrite('ReportImages/DS/Depth/small_new_depth.png', resize(new_depth[60:-20, :]))
    
    cv.destroyAllWindows()  # Deallocating memories taken for window creation

In [0]:
def saveDataAugmentation(image_path, depth_path, trajectories_path):
    
    num_of_points = 30
    
    with open(trajectories_path) as json_file:
        data = json.load(json_file)

    n_frame = int(image_path.split('/')[-1][-10:-4]) - 1
    frame = 'frame_{n}'.format(n='0' * (6 - len(str(n_frame))) + str(n_frame))
    
    small_image = cv.imread(image_path)
    small_depth = cv.imread(depth_path, -1)
    small_new_depth = np.array(small_depth)
    small_new_depth[small_new_depth > 10000] = 10000
     

    coorFuture = data[frame]["object_0"]["future"]
    coorFuture = coorFuture[0:num_of_points] if len(coorFuture) >= num_of_points else coorFuture # Vengono mostrati solo i primi 30 punti
    
    flip_small_image, flip_small_new_depth, flip_coorFuture = flip_img(small_image, small_new_depth, coorFuture)
    
    rb_small_image = randomBrightness(small_image)
    rb_flip_small_image = randomBrightness(flip_small_image)
    
    for i, item in enumerate(coorFuture):
      x = int(item[0])
      y = int(item[1])
      
      flip_x = flip_coorFuture[i][0]
      flip_y = flip_coorFuture[i][1]
      
      small_image[y, x] = [0, 255, 0]
      rb_small_image[y, x] = [0, 255, 0]
      flip_small_image[flip_y, flip_x] = [0, 255, 0]
      rb_flip_small_image[flip_y, flip_x] = [0, 255, 0]
      
    cv.imwrite('ReportImages/DA/Image/small_image.png', small_image)
    cv.imwrite('ReportImages/DA/Image/rb_small_image.png', rb_small_image)
    cv.imwrite('ReportImages/DA/Image/flip_small_image.png', flip_small_image)
    cv.imwrite('ReportImages/DA/Image/rb_flip_small_image.png', rb_flip_small_image)
    cv.imwrite('ReportImages/DA/Depth/small_depth.png', small_depth)
    cv.imwrite('ReportImages/DA/Depth/flip_small_depth.png', cv.flip(small_depth, 1))
    cv.imwrite('ReportImages/DA/Depth/small_new_depth.png', small_new_depth)
    cv.imwrite('ReportImages/DA/Depth/flip_small_new_depth.png', flip_small_new_depth)
    cv.destroyAllWindows()  # Deallocating memories taken for window creation

In [0]:
# IMMAGINI ORIGINALI
realTrajs_json = 'Sequenze-Depth-RGB/seq1/trajectories.json'
image = 'Sequenze-Depth-RGB/seq1/left/left001000.png'
depth = 'Sequenze-Depth-RGB/seq1/depth/depth001000.png'

# saveExamplesOfImages(image, depth, realTrajs_json)

# IMMAGINI PICCOLE
realTrajs_json = 'Dataset/IMG/train/seq1/trajectories.json'
image = 'Dataset/IMG/train/seq1/left/left001000.png'
depth = 'Dataset/IMG/train/seq1/depth/depth001000.png'

saveDataAugmentation(image, depth, realTrajs_json)



In [0]:
imgs_input = '* Test */test_flip/seq17/left/'

for image in sorted(os.listdir(imgs_input)):
  if (image.endswith('png')):  # Controlla che si tratti di un'immagine
    print(image)
    img = cv.imread(imgs_input + '{img_name}'.format(img_name=image), cv.IMREAD_UNCHANGED)
    img = randomBrightness(img)
    cv.imwrite('* Test */test/seq17/left/' + image, img)

In [0]:
# path = 'Sequenze-Depth-RGB/seq16/left/'
path = 'Sequenze-Depth-RGB/seq17/left/'

for i, image in enumerate(sorted(os.listdir(path))):
  if (image.endswith('png') and i == 1000):  # Controlla che si tratti di un'immagine
    print(image)
    img = cv.imread(path + '{img_name}'.format(img_name=image), cv.IMREAD_UNCHANGED)
    img = randomBrightness(img)
    img = cv.flip(img, 1)
    cv.imwrite(image, img)